In [ ]:
print("HI")

In [1]:
import pandas as pd
import numpy as np
import z2_save_jaspar
import z1_save_oligos
oligos,oligos_by_exp = z1_save_oligos.load_oligos()
jaspar = z2_save_jaspar.load_jaspar()

decent_motif_hits= pd.read_csv("../data/0712_motif_allregions_decenthits.csv")
all_regions = pd.read_csv("../data/0712_allregions.csv")


In [2]:
motifs_fewer_allbest = pd.read_csv("../data/0711_motif_fewerregions_allbest.csv").set_index(["seq","jaspar_id"]).assign(fewer_allbest=1)


In [ ]:
print("break the sequenced regions up into a partitioning of clusters, near and far from APOBECB")

from sklearn.cluster import MeanShift, estimate_bandwidth
import numpy as np

X_oligos = [[e] for e in oligos.starts]
bandwidth = estimate_bandwidth(X_oligos, quantile=0.05)
clustering = MeanShift(bandwidth).fit(X_oligos)
oligos_cluster_locs = clustering.labels_
oligos["cluster_partition"] = oligos_cluster_locs

#load up 60 bp regions selected in previous steps and compute a new set of regions of interest
many_regions_raw = pd.read_csv("../data/0711_manyregions.csv")[["seq","exp"]]
many_regions =pd.Series(many_regions_raw[["seq"]].groupby("seq").apply(lambda x: x.shape[0]).rename("count_across_exps").loc[lambda x:x>3].index)
fewer_regions = pd.read_csv("../data/0711_fewerregions.csv")[["seq"]].assign(exp=None).seq

pooled_regions = fewer_regions
replicable_regions = many_regions


all_regions = pd.merge(
    fewer_regions.to_frame().assign(found_in_pooled =True),
    replicable_regions.to_frame().assign(found_in_replicates=True),
    on="seq",how="outer")

#LOADS BIOLOGICAL MOTIFS AND SCANS ALL SUBREGIONS FOR OCCURENCES
from pyfaidx import Fasta
sequences_fa = Fasta('/cluster/bh0085/genomes/GRCh38.primary_assembly.genome.fa')
chrseq = str(sequences_fa["chr22"])

roi = [oligos.gstart.min(), oligos.gend.max()]
domainseq = chrseq[roi[0]:roi[1]]

byexp = many_regions_raw.assign(val=True).pivot(index="seq",columns="exp")['val']
region_multifactor_scores = byexp.replace([True],[1]).fillna(0).sum(axis = 1)

all_regions["start"] = all_regions.seq.apply(lambda x: domainseq.index(x))

obystart = oligos.reset_index().set_index(["starts","oligo"]).sort_index(level=0)
all_regions["partition"]= all_regions.start.apply(lambda s: obystart.loc[s-100 :s+100].cluster_partition.mode())
all_regions["pooled_mu"] = all_regions.start.apply(lambda s: obystart.loc[s-100 :s+100].mu.mean())
all_regions["wildtype_mu"] = all_regions.start.apply(lambda s: obystart.loc[s-100 :s+100].loc[lambda df: df.mutant_num==0].mu.mean())


In [5]:


all_regions = all_regions.filter(regex="^(?!(multi)).*$",axis="columns")
all_regions = all_regions.join(region_multifactor_scores.rename("multifactor_score"),on = "seq")
print(f"""{len(all_regions)} total regions, {len(region_multifactor_scores)} with multifactor scores, mean multifactor score = {all_regions.multifactor_score.mean()}""")

all_regions_w_exps = pd.merge(all_regions.set_index("seq"),
                                  byexp.fillna(False),
                               sort = True,
                              left_index=True,right_index=True,
                        )


print(f"""In 0702 and 0708 scripts, we compute "fewer" and "many" regions respectively
These have { len(many_regions) } and { len(fewer_regions) } regions each, of which the overlap is: 
{ len(set(fewer_regions).intersection(set(many_regions))) } regions. Henceforth, we can treat 
all of these as potential regions of interest
""")



448 total regions, 1185 with multifactor scores, mean multifactor score = 5.306532663316583
In 0702 and 0708 scripts, we compute "fewer" and "many" regions respectively
These have 289 and 242 regions each, of which the overlap is: 
83 regions. Henceforth, we can treat 
all of these as potential regions of interest



In [6]:


all_regions

,seq,found_in_pooled,found_in_replicates,start,partition,pooled_mu,wildtype_mu,multifactor_score
0,TGTCTCCTTTCCATGGCACACAACTCATGTTTTTTGTTTTAGGTTC...,True,NaN,225,13.0,19.378955,19.737948,NaN
1,AACACCCCACTCTTCCCTCTAGAGGAAGAATGTAAGCTTCTCAGTC...,True,True,285,13.0,15.289893,14.204168,14.0
2,GGGGCCCGCAGGGCGCGGCTCCCTCGGAAGGCGCGGATTCTAGCCC...,True,True,825,13.0,15.226862,16.515970,4.0
3,GCACAGGAAGACGTCGGGAAGACTGTGGAAGTGTCCTGGTGGAGGG...,True,NaN,18830,13.0,15.393644,11.822406,NaN
4,TCCCTAACAATCAGTTAGGAAACGCTCATCGGAAAGCATCCTTCCC...,True,True,55288,2.0,22.528072,25.156432,5.0
5,GGAAAGCATCCTTCCCCACTTCTCAGCTGGGCGACTTCCTTTCTCA...,True,True,55318,2.0,17.163675,19.081021,9.0
6,TGCTGTCTGGATCTGCAATTCTCAACATTTCCTCATGTCCGACACC...,True,True,88774,7.0,11.698742,15.892791,10.0
7,TGATCCGCCTGCCTCGGCCTCCCAAAGTGCTGGGATTACAGGCATG...,True,NaN,97845,7.0,23.783895,12.134662,1.0
8,GCCTATCTTGACCCCCTTTTCTTCTCACTAATGCTCTCAAAATTCT...,True,True,97935,7.0,14.203335,14.127278,10.0
9,GAGTCACTTGATCCTCATTCACTCAGGGCTCATAAGCCGAATACCT...,True,True,247725,0.0,13.644211,10.280988,5.0
